In [1]:
from my_weapon import *
from collections import defaultdict

In [2]:
url_tweets = json.load(open("data/url_tweets.json"))
# del url_tweets[0] # 第一条too general 目前的数据已经删除

starters_cnt = 0
first_users = set()
starter_users = set()

for fake in url_tweets:
    for j, t in enumerate(fake["tweets"]):
        if t["origin"]:
            starters_cnt += 1
            starter_users.add(t["user_id"])
            if j == 0:
                first_users.add(t["user_id"])
    
len(url_tweets), starters_cnt, len(first_users), len(starter_users)

(189620, 914272, 25212, 88215)

- 189620条谣言URL, 914272条初始推特；
- 25212个第一条谣言的用户, 88215个发初始推特的用户；

### 专门针对用户的分析，放到analyze_users.ipy中～

In [11]:
user_data = defaultdict(list)

for fake in tqdm(url_tweets):
    url = fake["url"]
    for j, t in enumerate(fake["tweets"]):
        uid = t["user_id"]
        if t["origin"]:
            if j == 0: # first
                user_data[str(uid)].append({"id": t["id"], "role": 0, "url": url, "dt": t["dt"]})
            else:
                user_data[str(uid)].append({"id": t["id"], "role": 1, "url": url, "dt": t["dt"]})
        else:
            user_data[str(uid)].append({"id": t["id"], "role": 2, "url": url, "dt": t["dt"]})
            

100%|██████████| 189620/189620 [00:02<00:00, 65628.35it/s]


In [26]:
# json.dump(tmp, open("data/user-behaviors.txt", "w"), indent=2)

user_data = json.load(open("data/user-behaviors.json"))

In [28]:
with open("data/user-behavior-features.csv", "w") as f:
    for d in user_data:
        uid = d["uid"]
        cnt_0 = 0
        cnt_1 = 0
        cnt_2 = 0
        url = set()
        for t in d["tweets"]:
            if t["role"] == 0:
                cnt_0 += 1
            elif t["role"] == 1:
                cnt_1 += 1
            elif t["role"] == 2:
                cnt_2 += 1
            url.add(t["url"])
        f.write("{},{},{},{},{},{}\n".format(uid, cnt_0, cnt_1, cnt_2, len(d["tweets"]), len(url)))
    

In [44]:
uids_of_IRA = set( [int(d) for d in json.load(open("data/IRA_match.json")).values()] )

In [31]:
user_data = pd.read_csv("data/user-behavior-features.csv")

In [48]:
cnt = 0

for i, row in user_data.iterrows():
    if row["uid"] in uids_of_IRA:
        print(row["uid"], row["cnt_url"])
        cnt += 1
        
cnt

789266125485998080 76
1679279490 61
1671234620 54
1662779209 18
4272870988 15
2606301939 15
743166519157227520 13
2491081213 13
2671070290 12
1694026190 11
2535166748 11
1661246144 10
1715424829 8
1651693646 9
1629828379 9
2572058134 7
1647045721 7
1649967228 7
4311371968 7
4224729994 6
1688370956 6
3990577513 5
2535818742 5
1833223908 5
3083844539 5
4355797094 5
4437444442 4
1533813962 4
1685718216 4
1710804738 4
1623180199 3
4289431230 3
1720117201 3
4224912857 3
4332740714 3
4208754922 3
1676481360 3
2540758755 3
4508630900 2
2570631118 2
4209684039 2
1655194147 2
1649144358 2
1690487623 2
2586506586 1
4218156466 1
1652987412 1
2533221819 1
1681162956 1
737420000043139072 1
1676377446 1
1659753702 1
3084866039 1
3312143142 1
1868981054 1
1617939218 1
1658202894 1
1653553555 1
1716228906 1


59